In [1]:
import numpy as np
import math
import torch
from string import punctuation
import torch.nn as nn
from ray import tune
import os

## Reading text files for reviews and sentiments

In [2]:
with open('data/reviews.txt') as f:
    reviews=f.read()
with open('data/labels.txt') as f:
    labels=f.read()

In [3]:
# make all words lowercase
reviews=reviews.lower()

#create reviews without punctuation and split it by new row
all_text=''.join(c for c in reviews if c not in punctuation)
all_text=all_text.split('\n')
final=''.join(all_text)

# make list of all words in text
final=final.split()


We have 6 milion words in text.

In [4]:
len(final)

6020196

In [5]:
final[:6]

['bromwell', 'high', 'is', 'a', 'cartoon', 'comedy']

## Tokenize words

Tokenize every word which appear in text to int value

In [6]:
unique_words=set(final)
pairs={w:i for (i,w) in enumerate(unique_words)}
pairs

{'burchill': 0,
 'ioana': 1,
 'concluding': 2,
 'chicken': 3,
 'bodden': 4,
 'conclude': 5,
 'parasite': 6,
 'kiwi': 7,
 'misinterpretated': 8,
 'handsaw': 9,
 'dainton': 10,
 'catfish': 11,
 'kirkendalls': 12,
 'kasumi': 13,
 'chriterion': 14,
 'flies': 15,
 'zp': 16,
 'diehl': 17,
 'snowed': 18,
 'helluva': 19,
 'ittenbach': 20,
 'goldstone': 21,
 'tatty': 22,
 'nagging': 23,
 'kaka': 24,
 'mikhail': 25,
 'northfork': 26,
 'doos': 27,
 'anjaane': 28,
 'reminiscences': 29,
 'smartens': 30,
 'dillusion': 31,
 'mandingo': 32,
 'shabnam': 33,
 'dwell': 34,
 'macguffin': 35,
 'laclos': 36,
 'yomiuri': 37,
 'scalp': 38,
 'unreasoned': 39,
 'addicts': 40,
 'boxing': 41,
 'debate': 42,
 'cloyingly': 43,
 'settleling': 44,
 'furdion': 45,
 'figga': 46,
 'castmates': 47,
 'validation': 48,
 'hinted': 49,
 'scuffle': 50,
 'erian': 51,
 'rattner': 52,
 'kinugasa': 53,
 'rosselinni': 54,
 'masochism': 55,
 'nombre': 56,
 'predates': 57,
 'nick': 58,
 'gradef': 59,
 'lumpens': 60,
 'exteremely': 6

## Tokenize every review

In [7]:
tokenized=[]
for review in all_text:
    tokenized.append([pairs[i] for (w,i) in enumerate(review.split())])
tokenized[:3]

[[27278,
  45081,
  64852,
  60680,
  60259,
  48651,
  18356,
  39551,
  29073,
  18670,
  56851,
  27772,
  59088,
  767,
  8999,
  60924,
  68248,
  29308,
  53015,
  72938,
  59088,
  73424,
  7520,
  63437,
  73188,
  18670,
  28861,
  20202,
  68308,
  65733,
  3036,
  4523,
  3110,
  27278,
  45081,
  45132,
  58772,
  64852,
  35658,
  13878,
  3036,
  68120,
  54858,
  64852,
  73424,
  18670,
  30966,
  3036,
  33193,
  16040,
  18670,
  10831,
  59906,
  24038,
  45782,
  29067,
  57905,
  58500,
  37047,
  41146,
  73424,
  11246,
  18670,
  60694,
  71987,
  18670,
  43293,
  21689,
  1889,
  53296,
  65733,
  71987,
  18670,
  32159,
  142,
  13795,
  56361,
  37047,
  59906,
  13085,
  142,
  41805,
  18670,
  68874,
  73188,
  50920,
  60680,
  22077,
  34490,
  54832,
  3036,
  25521,
  40917,
  18670,
  29308,
  142,
  3107,
  41818,
  29073,
  45081,
  60680,
  27791,
  18345,
  17539,
  142,
  22825,
  35969,
  3036,
  12420,
  7231,
  71987,
  10882,
  73424,
  220

In [8]:
labels_split=labels.split('\n')
encoded_labels=np.array([1 if l=='positive' else 0 for l in labels_split])

## Discard those reviews with zero length

In [9]:
from collections import Counter
lengths=Counter([len(x) for x in tokenized])
lengths[0]

1

In [10]:
for (i,t) in enumerate(tokenized):
    if(len(t)==0):
        del tokenized[i]
        encoded_labels=np.delete(encoded_labels,i)

In [11]:
lengths=Counter([len(x) for x in tokenized])
lengths[0]


0

## Clipping every review to array of fixed size
Clip array to fixed size length or if length of array is lower than fixed length for the input to LSTM, then fill the beginning of the array with zeros.

In [12]:
def fixed_size_features(tokenized,length):
    features=np.zeros((len(tokenized),length),dtype=int)
    for i,t in enumerate(tokenized):
        features[i,-len(t):]=np.array(t)[:length]
    return features


In [13]:
features=fixed_size_features(tokenized,200)

## Packing features to training, validation and test set

In [14]:
split_frac=0.8
idx=np.arange(len(features))
np.random.shuffle(idx)

train_len=int((split_frac*len(features)))
val_len=int(((1-split_frac)/2*len(features)))
train_x=features[idx[:train_len],:]
train_y=encoded_labels[idx[:train_len]]
val_x=features[idx[train_len:train_len+val_len+1],:]
val_y=encoded_labels[idx[train_len:train_len+val_len+1]]
test_x=features[idx[train_len+val_len+1:],:]
test_y=encoded_labels[idx[train_len+val_len+1:]]

In [15]:
from torch.utils.data import DataLoader,TensorDataset

trainset=TensorDataset(torch.from_numpy(train_x),torch.from_numpy(train_y))
testset=TensorDataset(torch.from_numpy(test_x),torch.from_numpy(test_y))
valset=TensorDataset(torch.from_numpy(val_x),torch.from_numpy(val_y))

batch_size=50

trainloader=DataLoader(trainset,batch_size=batch_size,shuffle=True)
testloader=DataLoader(testset,batch_size=batch_size,shuffle=True)
valloader=DataLoader(valset,batch_size=batch_size,shuffle=True)

In [16]:
# obtain one batch of training data
dataiter = iter(trainloader)
sample_x, sample_y = dataiter.next()

print('Sample input size: ', sample_x.size()) # batch_size, seq_length
print('Sample input: \n', sample_x)
print()
print('Sample label size: ', sample_y.size()) # batch_size
print('Sample label: \n', sample_y)

Sample input size:  torch.Size([50, 200])
Sample input: 
 tensor([[    0,     0,     0,  ..., 36303, 64852, 15485],
        [13085, 50084,  2637,  ..., 64852, 30898, 73188],
        [    0,     0,     0,  ..., 22871, 49755, 15797],
        ...,
        [42295, 22647, 53986,  ..., 26227, 53986, 53986],
        [69550, 54236, 63120,  ..., 28057, 64852, 71891],
        [    0,     0,     0,  ..., 71987, 35722, 16398]], dtype=torch.int32)

Sample label size:  torch.Size([50])
Sample label: 
 tensor([0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1,
        0, 0], dtype=torch.int32)


## Defining model

First we need to convert our words to embeddings which are input to LSTM. Every word in sentence is input to different LSTM and hidden layer is sent from previous LSTM to current LSTM as information about history. Also we use Softmax activation function to predict probability if the sentiment of the sentence. So, we are applying softmax to output of the last LSTM and that is the probability that sentence is positive or negative. 

In [17]:
import torch.nn as nn
class SentimentRNN(nn.Module):
    def __init__(self,vocab_size,hidden_size,output_size,n_layers,embedding_size,dropout_p=0.5):
        super(SentimentRNN,self).__init__()
        self.n_layers=n_layers
        self.hidden_size=hidden_size
        self.embedding=nn.Embedding(vocab_size,embedding_size)
        self.lstm=nn.LSTM(embedding_size,hidden_size,n_layers,batch_first=True,dropout=dropout_p)
        self.dropout=nn.Dropout(0.3)
        
        self.fc=nn.Linear(hidden_size,output_size)
        self.sigmoid=nn.Sigmoid()
        
    def forward(self,x,hidden):
        batch_size=x.size(0)
        x=self.embedding(x.long())
        r_out,hidden=self.lstm(x,hidden)
        
        r_out=r_out.contiguous().view(-1,self.hidden_size)
        out=self.fc(self.dropout(r_out))
        out=self.sigmoid(out)
        out=out.view(batch_size,-1)
        out=out[:,-1]
        return out,hidden
    
    def init_hidden(self,batch_size):
        weights = next(self.parameters()).data
        hidden=(weights.new(self.n_layers,batch_size,self.hidden_size).zero_(),
                      weights.new(self.n_layers,batch_size,self.hidden_size).zero_())
        return hidden
        

## Here is presented structure of our model

vocab_size is how many different words we have in our dataset

since we are predicting is the sentence positive or negative we have one output

n_layers is set to 2 which means that we have 2 stacked LSTMs

In [18]:
# Instantiate the model w/ hyperparams
vocab_size = len(unique_words)+1 # +1 for the 0 padding + our word tokens
output_size = 1
embedding_size = 400
n_layers = 2

## Training the model

Since we have to predict 2 classes, we are using Binary CrossEntropy Loss, here I use Adam optimizer with different learning rates which are written in config. I am using ray for hyperparameter tuning.

I tuned LSTM hyperparameters and also training hyperparameters.

In [19]:
def train_net(config,checkpoint_dir=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    net = SentimentRNN(vocab_size,hidden_size=config["hidden_dim"],output_size=output_size,n_layers=n_layers,embedding_size=config["emb_size"],dropout_p=config["dropout"])

    net.cuda()
    # training params
    epochs = config["epochs"] # 3-4 is approx where I noticed the validation loss stop decreasing

    counter = 0
    print_every = 100
    clip=5 # gradient clipping
    criterion=nn.BCELoss()
    optimizer=torch.optim.Adam(net.parameters(),lr=config["lr"])

    net.train()
    min_loss=math.inf
    min_val_acc=math.inf
    # train for some number of epochs
    for e in range(epochs):
        print(e)
        # initialize hidden state
        h = net.init_hidden(batch_size)

        # batch loop
        for inputs, labels in trainloader:
            counter += 1
            inputs,labels=inputs.cuda(),labels.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()

            # get the output from the model
            output, h = net(inputs, h)

            # calculate the loss and perform backprop
            loss = criterion(output.squeeze(), labels.float())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            optimizer.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                val_acc=[]
                corrects=0
                net.eval()
                for inputs, labels in valloader:
                    inputs,labels=inputs.cuda(),labels.cuda()
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, labels.float())
                    pred=torch.round(output)
                    correct_tensor = pred.eq(labels.float().view_as(pred))
                    correct = np.squeeze(correct_tensor.cpu().numpy()) 
                    corrects+=np.sum(correct)
                    val_losses.append(val_loss.item())
                    
                if(np.mean(val_losses)<min_loss):
                    min_loss=np.mean(val_losses)
                    min_val_acc=corrects/len(valloader.dataset)
#                     torch.save(net.state_dict(), 'model.pt')
                    with tune.checkpoint_dir(epochs) as checkpoint_dir:
                        path = os.path.join(checkpoint_dir, "checkpoint")
                        torch.save((net.state_dict(), optimizer.state_dict()), path)
                tune.report(train_loss=loss.item(),val_loss=min_loss)
                net.train()
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.6f}...".format(loss.item()),
                      "Val Loss: {:.6f}".format(np.mean(val_losses)))

In [20]:
config={
    "lr":tune.grid_search([0.001,0.01,0.1]),
    "epochs":tune.grid_search([3,5,7]),
  "dropout":tune.grid_search([0.2,0.3,0.5]),
    "emb_size":tune.grid_search([400]),
    "hidden_dim":tune.grid_search([100,256,500])
}

In [ ]:
analysis=tune.run(
    train_net,
    config=config,
    resources_per_trial={'gpu': 1}
    )

2020-12-04 20:24:23,304	INFO services.py:1090 -- View the Ray dashboard at http://127.0.0.1:8265
2020-12-04 20:24:33,518	ERROR syncer.py:63 -- Log sync requires rsync to be installed.
2020-12-04 20:24:33,914	WARNING worker.py:1091 -- Warning: The actor ImplicitFunc has size 18097973 when pickled. It will be stored in Redis, which could cause memory issues. This may mean that its definition uses a large array or other object.


Trial name,status,loc,dropout,emb_size,epochs,hidden_dim,lr
train_net_567cc_00000,RUNNING,,0.2,400,3,100,0.001


(pid=2184) 0
Result for train_net_567cc_00000:
  date: 2020-12-04_20-24-48
  done: false
  experiment_id: 08739d4650644243884af026b1c8dfd3
  experiment_tag: 0_dropout=0.2,emb_size=400,epochs=3,hidden_dim=100,lr=0.001
  hostname: DESKTOP-LTLCMDK
  iterations_since_restore: 1
  node_ip: 192.168.0.20
  pid: 2184
  should_checkpoint: true
  time_since_restore: 12.077111005783081
  time_this_iter_s: 12.077111005783081
  time_total_s: 12.077111005783081
  timestamp: 1607109888
  timesteps_since_restore: 0
  train_loss: 0.5978701710700989
  training_iteration: 1
  trial_id: 567cc_00000
  val_loss: 0.6418092906475067
  


Trial name,status,loc,dropout,emb_size,epochs,hidden_dim,lr,iter,total time (s),train_loss,val_loss
train_net_567cc_00000,RUNNING,192.168.0.20:2184,0.2,400,3,100,0.001,1,12.0771,0.59787,0.641809
train_net_567cc_00001,PENDING,,0.3,400,3,100,0.001,,,,


(pid=2184) Epoch: 1/3... Step: 100... Loss: 0.597870... Val Loss: 0.641809
Result for train_net_567cc_00000:
  date: 2020-12-04_20-24-56
  done: false
  experiment_id: 08739d4650644243884af026b1c8dfd3
  experiment_tag: 0_dropout=0.2,emb_size=400,epochs=3,hidden_dim=100,lr=0.001
  hostname: DESKTOP-LTLCMDK
  iterations_since_restore: 2
  node_ip: 192.168.0.20
  pid: 2184
  should_checkpoint: true
  time_since_restore: 19.992688179016113
  time_this_iter_s: 7.915577173233032
  time_total_s: 19.992688179016113
  timestamp: 1607109896
  timesteps_since_restore: 0
  train_loss: 0.6769163608551025
  training_iteration: 2
  trial_id: 567cc_00000
  val_loss: 0.609874633550644
  


...

(pid=1572) Epoch: 4/5... Step: 1300... Loss: 0.171960... Val Loss: 0.431094
Result for train_net_567cc_00005:
  date: 2020-12-04_20-36-11
  done: false
  experiment_id: 59e3a54075724cb0954cf947e5505f75
  experiment_tag: 5_dropout=0.5,emb_size=400,epochs=5,hidden_dim=100,lr=0.001
  hostname: DESKTOP-LTLCMDK
  iterations_since_restore: 14
  node_ip: 192.168.0.20
  pid: 1572
  time_since_restore: 106.94097113609314
  time_this_iter_s: 6.2224602699279785
  time_total_s: 106.94097113609314
  timestamp: 1607110571
  timesteps_since_restore: 0
  train_loss: 0.06964389234781265
  training_iteration: 14
  trial_id: 567cc_00005
  val_loss: 0.4239572310447693
  


Trial name,status,loc,dropout,emb_size,epochs,hidden_dim,lr,iter,total time (s),train_loss,val_loss
train_net_567cc_00005,RUNNING,192.168.0.20:1572,0.5,400,5,100,0.001,14,106.941,0.0696439,0.423957
train_net_567cc_00006,PENDING,,0.2,400,7,100,0.001,,,,
train_net_567cc_00000,TERMINATED,,0.2,400,3,100,0.001,12,92.9323,0.362514,0.428902
train_net_567cc_00001,TERMINATED,,0.3,400,3,100,0.001,12,90.64,0.31134,0.420858
train_net_567cc_00002,TERMINATED,,0.5,400,3,100,0.001,12,93.5948,0.241741,0.405401
train_net_567cc_00003,TERMINATED,,0.2,400,5,100,0.001,20,147.669,0.252086,0.421177
train_net_567cc_00004,TERMINATED,,0.3,400,5,100,0.001,20,146.67,0.19018,0.418606


(pid=1572) Epoch: 4/5... Step: 1400... Loss: 0.069644... Val Loss: 0.439467
Result for train_net_567cc_00005:
  date: 2020-12-04_20-36-18
  done: false
  experiment_id: 59e3a54075724cb0954cf947e5505f75
  experiment_tag: 5_dropout=0.5,emb_size=400,epochs=5,hidden_dim=100,lr=0.001
  hostname: DESKTOP-LTLCMDK
  iterations_since_restore: 15
  node_ip: 192.168.0.20
  pid: 1572
  time_since_restore: 113.16193318367004
  time_this_iter_s: 6.220962047576904
  time_total_s: 113.16193318367004
  timestamp: 1607110578
  timesteps_since_restore: 0
  train_loss: 0.2389502227306366
  training_iteration: 15
  trial_id: 567cc_00005
  val_loss: 0.4239572310447693
  


Trial name,status,loc,dropout,emb_size,epochs,hidden_dim,lr,iter,total time (s),train_loss,val_loss
train_net_567cc_00005,RUNNING,192.168.0.20:1572,0.5,400,5,100,0.001,15,113.162,0.23895,0.423957
train_net_567cc_00006,PENDING,,0.2,400,7,100,0.001,,,,
train_net_567cc_00000,TERMINATED,,0.2,400,3,100,0.001,12,92.9323,0.362514,0.428902
train_net_567cc_00001,TERMINATED,,0.3,400,3,100,0.001,12,90.64,0.31134,0.420858
train_net_567cc_00002,TERMINATED,,0.5,400,3,100,0.001,12,93.5948,0.241741,0.405401
train_net_567cc_00003,TERMINATED,,0.2,400,5,100,0.001,20,147.669,0.252086,0.421177
train_net_567cc_00004,TERMINATED,,0.3,400,5,100,0.001,20,146.67,0.19018,0.418606


(pid=1572) Epoch: 4/5... Step: 1500... Loss: 0.238950... Val Loss: 0.442156


Getting config with best minimum mean loss on validation set

In [ ]:
best_conf=analysis.get_best_config(metric="mean_loss", mode="min")

## Loading saved network with optimal parameters

Making constructor for network with optimal parameters and then loding best saved checkpoint for that model.

In [ ]:
best_trial = analysis.get_best_trial("mean_loss", "min", "last")
best_trained_model = (best_trial.config["lr"])    
best_trained_model=SentimentRNN(vocab_size,hidden_dim,output_size,n_layers,embedding_size)#

best_checkpoint_dir = best_trial.checkpoint.value
model_state, optimizer_state = torch.load(os.path.join(
    best_checkpoint_dir, "checkpoint"))
best_trained_model.load_state_dict(model_state)


## Testing the network

In [ ]:
def test(model):

    # Get test data loss and accuracy

    test_losses = [] # track loss
    num_correct = 0
    net=model.cuda()
    criterion=nn.BCELoss()
    optimizer=torch.optim.Adam(net.parameters(),lr=best_trial.config["lr"])
    # init hidden state
    h = net.init_hidden(batch_size)

    net.eval()
    # iterate over test data
    for inputs, labels in testloader:
        inputs,labels=inputs.cuda(),labels.cuda()
        # Creating new variables for the hidden state, otherwise
        # we'd backprop through the entire training history
        h = tuple([each.data for each in h])

        # get predicted outputs
        output, h = net(inputs, h)

        # calculate loss
        test_loss = criterion(output.squeeze(), labels.float())
        test_losses.append(test_loss.item())

        # convert output probabilities to predicted class (0 or 1)
        pred = torch.round(output.squeeze())  # rounds to the nearest integer

        # compare predictions to true label
        correct_tensor = pred.eq(labels.float().view_as(pred))
        correct = np.squeeze(correct_tensor.cpu().numpy()) 
        num_correct += np.sum(correct)


    # -- stats! -- ##
    # avg test loss
    print("Test loss: {:.3f}".format(np.mean(test_losses)))

    # accuracy over all test data
    test_acc = num_correct/len(testloader.dataset)
    print("Test accuracy: {:.3f}".format(test_acc))

In [ ]:
test(best_trained_model)

## We are taking our example of review and testing it on LSTM

First we need to preprocess our sentence, tokenize it, convert to Tensor and then forward it to the input of the LSTM to make sentiment prediction of the sentence.

In [ ]:
test_review_pos = 'This movie had the best acting and the dialogue was so good. I loved it.'


In [ ]:
def prepare_for_network(text, sequence_length):
    text=text.lower()
    text_without_punctuation=''.join([c for c in text if c not in punctuation])
    test_words=text_without_punctuation.split()
    test_ints=[]
    test_ints.append([pairs[w] for w in test_words])
    test_input=fixed_size_features(test_ints,sequence_length)
    return test_input

In [ ]:
inputs=prepare_for_network(test_review_pos,200)

In [ ]:
inputs

In [ ]:
def predict_sentiment(net,sequence):
    seq=torch.tensor(sequence)
    batch_size=seq.shape[0]
    h=net.init_hidden(batch_size)
    seq=seq.cuda()
    out,h=net(seq,h)
    print(out.item())
    pred=round(out.item())
    if(pred==1.0):
        print("Positive")
    else:
        print("Negative")

In [ ]:
predict_sentiment(net,inputs)

In [ ]:
test_review_neg = "This movie had terrible actors, I didn't like it at all."
inputs1=prepare_for_network(test_review_neg,200)

In [ ]:
def predict_sentiment(net,inputs,hidden=None):
    out=net(inputs.squeeze(),hidden)
    

In [ ]:
predict_sentiment(net,inputs1)

In [ ]:
predict_sentiment(net,inputs)